# A simple iterative program using sessions
This is a simple example showing how we find the theta which minimizes the Mermin operator 

$$ 
M = XXY+XYX+YXX-YYY
$$  

for a search over quantum states 

$$ 
|\psi\rangle = [|000\rangle  + e^{i \theta} |111\rangle ]/\sqrt{2}
$$ 


## **Step 0**: Setup

In [ ]:
import numpy as np

# Qiskit Quantum Circuit
from qiskit import QuantumCircuit

# Qiskit Operator form
from qiskit.quantum_info import SparsePauliOp

# Import Qiskit packages
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options, Session

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum", instance="executive/internal/main")

# Define backend
use_real = True

if use_real:
    backend = service.backend('ibm_kyiv')
else:
    backend = service.backend('ibmq_qasm_simulator')

## **Step 1** Map the problem to a Quantum Native format (Set of Operators, and a set of Quantum Circuits)
Here we define the Mermin operator to be measured and the trial quantum circuits to be prepared to find the quantum circuit that maximizes the Mermin operator. The output of this step should be an operator to be measured and a quantum circuit

In [ ]:
mermin = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])
print(mermin)

In [ ]:
from qiskit.circuit import Parameter
theta = Parameter('θ')


# Step 1. Choose the quantum circuit to make the quantum state |000> + e^{theta} |111>
qc_example = QuantumCircuit(3)
qc_example.h(0) # generate superposition
qc_example.p(theta, 0) # add quantum phase
qc_example.cx(0, 1) # condition 1st qubit on 0th qubit
qc_example.cx(0, 2) # condition 2nd qubit on 0th qubit
print(qc_example)

## **Step 2**: Optimize the circuits and the operators to be measured

In [4]:
from qiskit.compiler import transpile
qc_ibm = transpile(qc_example, basis_gates = ['cz', 'sx', 'rz'],  coupling_map =[[0, 1], [1, 2]], optimization_level=3)
print(qc_ibm)

         ┌─────────┐┌────┐ ┌───────┐                      ┌────┐  ┌─────────┐
q_2 -> 0 ┤ Rz(π/2) ├┤ √X ├─┤ Rz(π) ├───────────────■──────┤ √X ├──┤ Rz(π/2) ├
         ├─────────┤├────┤┌┴───────┴┐┌───────┐     │      └────┘  └─────────┘
q_0 -> 1 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ Rz(θ) ├─■───■─────────────────────────
         ├─────────┤├────┤└┬───────┬┘└───────┘ │ ┌────┐┌─────────┐           
q_1 -> 2 ┤ Rz(π/2) ├┤ √X ├─┤ Rz(π) ├───────────■─┤ √X ├┤ Rz(π/2) ├───────────
         └─────────┘└────┘ └───────┘             └────┘└─────────┘           


In [5]:
from permute_sparse_pauli_op import permute_sparse_pauli_op

layout = qc_ibm.layout.initial_layout
mermin_ibm = permute_sparse_pauli_op(mermin,layout, qc_example.qubits)
mermin_ibm

SparsePauliOp(['XYX', 'YXX', 'XXY', 'YYY'],
              coeffs=[ 1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j])

## **Step 3**: Execute using a quantum primitive function (estimator or sampler)

In [6]:
options = Options(optimization_level=1)
options.execution.shots = 5000  # Options can be set using auto-complete.

# golden search method for finding max of M1 vs theta 
# https://en.wikipedia.org/wiki/Golden-section_search
gr = (np.sqrt(5) + 1) / 2

# range of theta  
thetaa = 0
thetab = 2*np.pi

#tol 
tol = 1e-1

with Session(backend=backend) as session:
    estimator = Estimator(options=options)

    #next test range 
    thetac = thetab - (thetab - thetaa) / gr
    thetad = thetaa + (thetab - thetaa) / gr
    while abs(thetab - thetaa) > tol:
        
        print(f"min value of M1 is in the range theta = {[thetaa, thetab]}")
        job = estimator.run(circuits=[qc_ibm]*2, observables=[mermin_ibm]*2, parameter_values=[[thetac],[thetad]])
        
        test =job.result().values
        if test[0] < test[1]:
            thetab = thetad
        else:
            thetaa = thetac
        
        thetac = thetab - (thetab - thetaa) / gr
        thetad = thetaa + (thetab - thetaa) / gr
        
    # Final Job to evaluate estimator at mid point found using golden search method 
    theta_mid = (thetab + thetaa) / 2
    job = estimator.run(circuits=qc_ibm, observables=mermin_ibm, parameter_values=theta_mid)
    print(f"Session ID is {session.session_id}")
    print(f"Final Job ID is {job.job_id()}")

min value of M1 is in the range theta = [0, 6.283185307179586]
min value of M1 is in the range theta = [2.3999632297286535, 6.283185307179586]
min value of M1 is in the range theta = [3.883222077450933, 6.283185307179586]
min value of M1 is in the range theta = [3.883222077450933, 5.366480925173213]
min value of M1 is in the range theta = [4.4497765431668395, 5.366480925173213]
min value of M1 is in the range theta = [4.4497765431668395, 5.016331008882746]
min value of M1 is in the range theta = [4.4497765431668395, 4.799926459457307]
min value of M1 is in the range theta = [4.583521910031868, 4.799926459457307]
min value of M1 is in the range theta = [4.583521910031868, 4.717267276896896]
Session ID is ck0hjg6iel5ovfa3ndeg
Final Job ID is ck0hrrmiel5ovfa46q8g


## **Step 4**: Post-processing of the results to return either a plot or the answer

In [7]:
print(f"Job result is {job.result().values} at theta = {theta_mid}")

Job result is [-3.86278234] at theta = 4.67593768561669


In [8]:
print(f"In units of pi theta = {theta_mid/np.pi}")

In units of pi theta = 1.488397192511146
